<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a> | <a href="https://chat.vlm.run"><b>Chat</b></a>
</p>
</div>

# VLM Run Orion - Radiology Report Draft

This notebook demonstrates how to use [VLM Run Orion's](https://vlm.run/orion) vision capabilities to act as a Radiology Assistant. You can provide a chest X-ray (CXR) and ask the model to draft a preliminary observation report, structured with specific medical sections.

For more details on the API, see the [Agent API docs](https://docs.vlm.run/agents/introduction).

## Prerequisites

- Python 3.10+
- VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
- VLM Run Python Client with OpenAI extra `vlmrun[openai]`

## Setup

First, install the required packages and configure the environment.

In [1]:
%pip install vlmrun[openai] --upgrade --quiet
%pip install cachetools pillow requests numpy --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import getpass

VLMRUN_API_KEY = os.getenv("VLMRUN_API_KEY", None)
if VLMRUN_API_KEY is None:
    VLMRUN_API_KEY = getpass.getpass("Enter your VLM Run API key: ")

## Initialize the VLM Run Client

We use the OpenAI-compatible chat completions interface through the VLM Run SDK.

In [3]:
from vlmrun.client import VLMRun

BASE_URL = os.getenv("VLMRUN_BASE_URL", "https://agent.vlm.run/v1")
client = VLMRun(api_key=VLMRUN_API_KEY, base_url=BASE_URL)
print("VLM Run client initialized successfully!")
print(f"Base URL: {BASE_URL}")

VLM Run client initialized successfully!
Base URL: https://agent.vlm.run/v1


## Response Models

We define Pydantic models for structured outputs. The report will be returned as a structured object containing specific sections like Technical Quality, Systematic Review, Findings, and Impression.

In [4]:
from typing import List
from pydantic import BaseModel, Field


class SystematicReview(BaseModel):
    """Detailed observations for specific anatomical landmarks."""
    lungs: str = Field(..., description="Observations detailed regarding opacities, nodules, masses, or signs of consolidation/effusion")
    pleura: str = Field(..., description="Observations regarding signs of pneumothorax or pleural thickening")
    heart_mediastinum: str = Field(..., description="Assessment of cardiothoracic ratio (CTR) and the contours of the mediastinum and hila")
    diaphragm_costophrenic_angles: str = Field(..., description="Visual assessment of diaphragm and costophrenic angles")
    bony_structures_soft_tissues: str = Field(..., description="Identification of any fractures, lesions, or surgical hardware")


class RadiologyReport(BaseModel):
    """Structured preliminary observation report for a Chest X-ray."""
    technical_quality: str = Field(..., description="Assessment of image quality (projection, inspiration, rotation, penetration)")
    systematic_review: SystematicReview = Field(..., description="Systematic review of anatomical landmarks")
    findings_summary: List[str] = Field(..., description="List of significant observations in a clear, concise format")
    impression: str = Field(..., description="Summary of the most likely clinical interpretations based solely on visual evidence")

print("Response models defined successfully!")

Response models defined successfully!


## Helper Functions

We create helper functions to simplify making chat completion requests with structured outputs.

In [5]:
import hashlib
import json
from typing import Any, Type, TypeVar

import cachetools
from vlmrun.common.image import encode_image
from PIL import Image


T = TypeVar('T', bound=BaseModel)


def custom_key(prompt: str, image_path: str | None = None, response_model: Type[T] | None = None, model: str = "vlmrun-orion-1:auto"):
    """Custom key for caching chat_completion."""
    response_key = hashlib.sha256(json.dumps(response_model.model_json_schema(), sort_keys=True).encode()).hexdigest() if response_model else ""
    image_key = hashlib.sha256(image_path.encode()).hexdigest() if image_path else ""
    return (prompt, image_key, response_key, model)


@cachetools.cached(cache=cachetools.TTLCache(maxsize=100, ttl=3600), key=custom_key)
def chat_completion(
    prompt: str,
    image_path: str | None = None,
    response_model: Type[T] | None = None,
    model: str = "vlmrun-orion-1:auto"
) -> tuple[BaseModel | str, str]:
    """
    Make a chat completion request with structured output for Radiology Reporting.

    Args:
        prompt: The prompt describing the reporting task
        image_path: Path to the image file (Chest X-ray)
        response_model: Pydantic model for structured output
        model: Model to use (default: vlmrun-orion-1:auto)

    Returns:
        Tuple of (parsed response model or text, session_id)
    """
    content = [{"type": "text", "text": prompt}]

    if image_path:
        image = Image.open(image_path)
        image_data = encode_image(image, format="JPEG")
        content.append({"type": "image_url", "image_url": {"url": image_data}})

    kwargs = {
        "model": model,
        "messages": [{"role": "user", "content": content}]
    }

    if response_model:
        kwargs["response_format"] = {
            "type": "json_schema",
            "schema": response_model.model_json_schema()
        }

    response = client.agent.completions.create(**kwargs)
    response_text = response.choices[0].message.content
    session_id = response.session_id

    if response_model:
        result = response_model.model_validate_json(response_text)
        return result, session_id

    return response_text, session_id

print("Helper functions defined!")

Helper functions defined!


## Radiology Report Generation

Provide a chest X-ray and ask the VLM to draft a structured preliminary observation report.

In [6]:
RADIOLOGY_PROMPT = """
Task: Act as a Radiology Assistant. Analyze the provided chest X-ray (CXR) and draft a structured preliminary observation report.

Instructions:

Technical Quality Assessment: Briefly comment on the image quality (e.g., projection, inspiration, rotation, and penetration).

Systematic Review: Provide observations based on the following anatomical landmarks:

Lungs: Check for opacities, nodules, masses, or signs of consolidation/effusion.

Pleura: Note any signs of pneumothorax or pleural thickening.

Heart and Mediastinum: Assess cardiothoracic ratio (CTR) and the contours of the mediastinum and hila.

Diaphragm and Costophrenic Angles: Note if the angles are sharp or obscured.

Bony Structures and Soft Tissues: Identify any fractures, lesions, or surgical hardware.

Findings Summary: List significant observations in a clear, concise bulleted format.

Impression: Provide a brief summary of the most likely clinical interpretations based solely on the visual evidence.

Refinement: Use professional medical terminology (e.g., "radiopaque," "radiolucent," "perihilar," "atelectasis") where appropriate.
"""

print("Radiology prompt prepared!")
print(f"\\nPrompt length: {len(RADIOLOGY_PROMPT)} characters")

Radiology prompt prepared!
\nPrompt length: 1114 characters


In [ ]:
image_path = "x-ray.jpeg" 

if os.path.exists(image_path):
    result, session_id = chat_completion(
        prompt=RADIOLOGY_PROMPT,
        image_path=image_path,
        response_model=RadiologyReport,
        model="vlmrun-orion-1:auto"
    )

    print(">> RESPONSE")
    print(result)
    print(f"\\n>> SESSION ID: {session_id}")
    
    print("\\n>> GENERATED RADIOLOGY REPORT")
    print("=" * 80)
    print(f"Technical Quality: {result.technical_quality}\\n")
    print("Systematic Review:")
    print(f"- Lungs: {result.systematic_review.lungs}")
    print(f"- Pleura: {result.systematic_review.pleura}")
    print(f"- Heart & Mediastinum: {result.systematic_review.heart_mediastinum}")
    print(f"- Diaphragm & Angles: {result.systematic_review.diaphragm_costophrenic_angles}")
    print(f"- Bones & Soft Tissues: {result.systematic_review.bony_structures_soft_tissues}\\n")
    print("Findings Summary:")
    for finding in result.findings_summary:
        print(f"- {finding}")
    print(f"\\nImpression:\\n{result.impression}")
    print("=" * 80)
else:
    print(f"Image not found at {image_path}. Please add a chest X-ray image to run this example.")

>> RESPONSE
technical_quality='Adequate PA projection. Inspiration is good (9-10 posterior ribs). No significant rotation. Penetration is adequate, with vertebral bodies visible behind the heart.' systematic_review=SystematicReview(lungs='Lung fields are clear bilaterally with normal bronchovascular markings. No radiopaque opacities, nodules, or masses are seen. The green highlighted area is an image annotation.', pleura='No evidence of pneumothorax or significant pleural thickening is identified.', heart_mediastinum='The cardiothoracic ratio (CTR) is within normal limits. Mediastinal and hilar contours are unremarkable with no evidence of widening or masses.', diaphragm_costophrenic_angles='Diaphragmatic domes are smooth and well-defined. Costophrenic angles are sharp and clear bilaterally.', bony_structures_soft_tissues='The ribs, clavicles, scapulae, and visualized vertebrae appear intact without acute fractures or destructive lesions. Soft tissues are unremarkable; no surgical hard

---

## Conclusion

This notebook demonstrated how to use **VLM Run Orion** to draft structured radiology reports from medical imaging.

### Key Takeaways

1. **Structured Outputs**: Using Pydantic models allows for highly structured, consistent medical reporting.
2. **Domain Specificity**: The model can interpret medical terminology and anatomical structures when prompts are well-crafted.
3. **Workflow Integration**: This automated drafting can serve as a "second pair of eyes" or a starting point for radiologists.

### Next Steps

- Test with different modalities (CT, MRI).
- Refine the Pydantic models to include specific flags (e.g., `has_pneumothorax: bool`).
- Explore the [VLM Run Documentation](https://docs.vlm.run) for more capabilities.

Happy analyzing! 🩺